In [3]:
import pandas as pd
import numpy as np
import json
import sys

In [3]:
df = pd.read_json("objects.json", dtype={"name":str, "coords": np.array})
#df["coords"] = df.coords.astype(np.ndarray)

In [4]:
df.head()

,coords,name
0,"[-107.837519201, 107.752265664, -22.7960154139...",5i4qyrl6chofxhwrsllzjqhishtb417anp9ld3mo2b997x...
1,"[145.280255119, 138.388473321, -118.932948485,...",rexb52du65nl8da51clyi3rfdrvatd5t9ryez8eozuw0d3...
2,"[125.348218698, 71.7519941445, 65.2878742037, ...",4yevon0tc78ipgcezvpjkc4id1pqkqoyw005gpnq635zou...
3,"[-9.1556327249, -32.5038742627, 83.0107244253,...",v6i56wa5snigfknxia7dlf5vrdbl88sbhbx9w1x8l54sgu...
4,"[-81.2724350109, 134.760054097, -12.5143615838...",6yxge1kgn53ow9z5ojm5v14iaqlv8d1mwbtowvt4egf048...


In [18]:
df.shape

(200000, 2)

In [5]:
df.coords.map(lambda l: [len(str(x)) for x in l]).map(np.mean).mean()

13.395049350385172

In [5]:
lens = df.coords.map(len) * 9 + df.name.map(len)

In [47]:
l = [1045, 1144, 1247, 1351, 1450]
div = []
for i in range(1,5):
    div.append(l[i] * 1.0 / l[i-1])
print div
fact = np.mean(div)

[1.0947368421052632, 1.090034965034965, 1.083400160384924, 1.073279052553664]


In [6]:
lens.describe()

count    200000.000000
mean       1247.254530
std         588.869052
min          47.000000
25%         746.000000
50%        1247.000000
75%        1745.000000
max        2450.000000
dtype: float64

In [19]:
factors = np.linspace(1.01, 1.5, 50)
min_err = np.inf
arg_fac_min = 1.05
MIN_LEN = 50
SIZE = lens.shape[0] # for normalization of errors
MAX_LEN = 2450
coeff = 5 # fine for big number of bins
func = np.vectorize(lambda x: x if x >= 0 else np.inf)
for factor in factors:
    err = 0.0
    bins = [MIN_LEN]
    f = MIN_LEN
    while f < MAX_LEN:
        f *= factor
        bins.append(int(f))
    binsa = np.array(bins)
    for length in lens:
        err += (binsa[binsa >= length].min() - length) * 1.0 / SIZE 
    err += coeff * len(bins)
    print "factor: %s, err: %s, bins: %s" % (factor, err, len(bins))
    if err < min_err:
        arg_fac_min = factor
        min_err = err

arg_fac_min     

factor: 1.01, err: 1970.73278, bins: 393
factor: 1.02, err: 1001.935415, bins: 198
factor: 1.03, err: 683.144025, bins: 133
factor: 1.04, err: 529.24004, bins: 101
factor: 1.05, err: 435.36874, bins: 81
factor: 1.06, err: 376.568765, bins: 68
factor: 1.07, err: 337.68323, bins: 59
factor: 1.08, err: 308.911705, bins: 52
factor: 1.09, err: 289.72668, bins: 47
factor: 1.1, err: 270.995455, bins: 42
factor: 1.11, err: 261.758465, bins: 39
factor: 1.12, err: 252.65348, bins: 36
factor: 1.13, err: 244.29616, bins: 33
factor: 1.14, err: 240.28285, bins: 31
factor: 1.15, err: 235.99539, bins: 29
factor: 1.16, err: 236.517575, bins: 28
factor: 1.17, err: 233.03078, bins: 26
factor: 1.18, err: 233.78421, bins: 25
factor: 1.19, err: 234.532255, bins: 24
factor: 1.2, err: 234.92727, bins: 23
factor: 1.21, err: 235.76589, bins: 22
factor: 1.22, err: 237.02485, bins: 21
factor: 1.23, err: 239.057245, bins: 20
factor: 1.24, err: 243.096145, bins: 20
factor: 1.25, err: 243.36887, bins: 19
factor: 1.2

1.1699999999999999

In [24]:
def find_factor(sizes, space, bins_fine):
    min_err = np.inf
    arg_fac_min = space[0]
    MIN_LEN = sizes.min()
    SIZE = sizes.shape[0] # for normalization of errors
    MAX_LEN = sizes.max()
    for factor in factors:
        err = 0.0
        bins = [MIN_LEN]
        f = MIN_LEN
        while f < MAX_LEN:
            f *= factor
            bins.append(int(f))
        bins = np.array(bins)
        for length in lens:
            err += (bins[bins >= length].min() - length) * 1.0 / SIZE 
        err += bins_fine * len(bins)
        #print "factor: %s, err: %s, bins: %s" % (factor, err, len(bins))
        if err < min_err:
            arg_fac_min = factor
            min_err = err
    return arg_fac_min, min_err

In [26]:
find_factor(lens, np.linspace(1.01, 1.5, 50), 3)

(1.1400000000000001, 180.42321000001226)

# Here we get statistics of beer dataset

In [4]:
df = pd.read_csv('beer_subset.csv.gz', parse_dates=['time'], compression='gzip')

In [4]:
df.head()

,abv,beer_id,brewer_id,beer_name,beer_style,review_appearance,review_aroma,review_overall,review_palate,profile_name,review_taste,text,time
0,7.0,2511,287,Bell's Cherry Stout,American Stout,4.5,4.0,4.5,4.0,blaheath,4.5,Batch 8144\tPitch black in color with a 1/2 f...,2009-10-05 21:31:48
1,5.7,19736,9790,Duck-Rabbit Porter,American Porter,4.5,4.0,4.5,4.0,GJ40,4.0,Sampled from a 12oz bottle in a standard pint...,2009-10-05 21:32:09
2,4.8,11098,3182,Fürstenberg Premium Pilsener,German Pilsener,4.0,3.0,3.0,3.0,biegaman,3.5,Haystack yellow with an energetic group of bu...,2009-10-05 21:32:13
3,9.5,28577,3818,Unearthly (Imperial India Pale Ale),American Double / Imperial IPA,4.0,4.0,4.0,4.0,nick76,4.0,"The aroma has pine, wood, citrus, caramel, an...",2009-10-05 21:32:37
4,5.8,398,119,Wolaver's Pale Ale,American Pale Ale (APA),4.0,3.0,4.0,3.5,champ103,3.0,A: Pours a slightly hazy golden/orange color....,2009-10-05 21:33:14


In [49]:
length = 0
df.time = df.time.astype(int)
for name in df.columns:
    length += df[name].map(lambda x: str(x)).map(len).sum()
length* 1.0 /(1024 * 1024)

76.740906715393066

In [5]:
df.dropna(inplace=True)

In [6]:
df.text.map(len).mean()

737.30289304801795

In [54]:
import zlib
df.text.map(zlib.compress).map(len).mean()

427.38948845463364

In [42]:
a = df.time.map(lambda x: str(x)).map(len)

In [12]:
import redis
r = redis.Redis(host='localhost', port=6379, db=0)

"[5.9, 36034, 624, '6288 Stout', 'American Stout', 4.0, 4.0, 4.0, 4.0, 'mikesgroove', 4.0, ' I am really not sure where this one came from, but I apparently picked it up recently and saw it sitting in the fridge so I grabbed for it last night while watching the game. Served chilled and poured into a pint glass this one was consumed on 10/20/2009. \\t \\tReally nice pour, very typical for the style with a deep dark black look to it and a nice head of maybe two inches that came up and over the top with some ease. Settling down it left a decent amount of lacing on the sides, but nothing spectacular though it did retain a good bit of the puff of tan on top. The aroma was nice, well blended mix of chocolate and roasted malts, hints of light coffee notes and a decent bit of dark fruit note carrying over. This was pretty much par for the course with the flavor. Started out very smooth with a rich and well defined chocolate note that dominated early before mellowing with a nice but of vanilla 

In [50]:
s = 0
for i, val in enumerate(df.values):
    s += len(r.get(i))
s

84477909

In [51]:
s * 1.0 /(1024 * 1024)

80.5644121170044

In [17]:
s = r.get(5000)

In [29]:
import pymongo
client = pymongo.MongoClient()
db = client.test2
res = db.command("collstats", "nosql")
res

{u'avgObjSize': 733,
 u'capped': False,
 u'count': 96922,
 u'indexDetails': {u'_id_': {u'LSM': {u'bloom filter false positives': 0,
    u'bloom filter hits': 0,
    u'bloom filter misses': 0,
    u'bloom filter pages evicted from cache': 0,
    u'bloom filter pages read into cache': 0,
    u'bloom filters in the LSM tree': 0,
    u'chunks in the LSM tree': 0,
    u'highest merge generation in the LSM tree': 0,
    u'queries that could have benefited from a Bloom filter that did not exist': 0,
    u'sleep for LSM checkpoint throttle': 0,
    u'sleep for LSM merge throttle': 0,
    u'total size of bloom filters': 0},
   u'block-manager': {u'allocations requiring file extension': 66,
    u'blocks allocated': 66,
    u'blocks freed': 0,
    u'checkpoint size': 782336,
    u'file allocation unit size': 4096,
    u'file bytes available for reuse': 0,
    u'file magic number': 120897,
    u'file major version number': 1,
    u'file size in bytes': 794624,
    u'minor version number': 0},
   u

In [22]:
import zlib
bytes(zlib.compress(df.text[0]))

'x\x9cMP1n\x031\x0c\x9b\xfb\x0bn]\x0e\x17\xa4\xc8\xd0\xb5\xdd2\x14\xe8\xd0\x0f(\xb6\x1c\x0b\xb1O\x81\xad\xc3\xe1~_9h\xd1.\x82(\x81\xa4(\xbc\x93\x85\x8c\xd7\xe3\xe9\xf4\xf4)\xa3\xbd\x14\n7\xc8\x82\xa0E\x1b6\xb1\x0c\xc2\xf1\xf0\x82$\xcb\x95\x1b2S\x9c\xf1\xd6\xb4\x12\xa4C\x13"\xb5\x1bR[\xc5\xfa\x84\x90\xd5\x99d<\x81\x96\x88\xa6\xd4\x8d#*\x15\xeb3\xbe\x06\x1a\xb4\xa8U\x16\x1a\x9b\xcb\x0e\xa3fN\xe4\xd6\xf6_\xc3"\xd7l\xa0 Ql\xc7\xbd\xac\xfdO\xf9\xf0_\xf4\xe1\x124%\xe6\x19\x1f\x1ce\xad\xa8\xbaZ\xf6Ay,\t=\xab\x1b\xf4\x8d\xd9F\n\xe9\xd9\x03\x18lS\xecL\xcdC\x948\xe1\x0c\xcb\xb2\xdc\xbc\x92\x8d\xb6\xa3\x9b\x94\x82L\xdd!;\xa2\xdd_\x80\xbbn\xfe\x86\xe4\xcf!$\xde~$L\xfd\x8a\xea7\x9c\x11hy6l$6\xa6n\x18\xe1\x07\xcd\xdf\x1b\xde\x7f\xab'

In [23]:
d = df.to_dict(orient='records')

In [24]:
str(d[0])

'{\'text\': " Batch 8144\\tPitch black in color with a 1/2 finger head. Aroma is of dark fruits, chocolate, and roasted malts. Taste is dominated by tart cherry with a light acidity plus chocolate/roasted malt and coffee. Medium mouthfeel and a short sweet finish. At two years old, I think that this still has the staying power for a few years to come. I can\'t wait to find out.", \'review_overall\': 4.5, \'review_aroma\': 4.0, \'review_appearance\': 4.5, \'beer_style\': \'American Stout\', \'review_palate\': 4.0, \'review_taste\': 4.5, \'beer_name\': "Bell\'s Cherry Stout", \'abv\': 7.0, \'time\': Timestamp(\'2009-10-05 21:31:48\'), \'beer_id\': 2511, \'brewer_id\': 287, \'profile_name\': \'blaheath\'}'

In [30]:
from pymemcache.client.base import Client
mc = Client(('localhost', 11211))


In [32]:
mc.stats()["bytes"] * 1.0 / (1024 * 1024)

105.15503883361816

In [18]:
112912145 * 1.0 / (1024 * 1024)

107.68141269683838

In [11]:
from couchdb.client import Server
server = Server()
db = server.create("test")
#db.save({"name":"gogogog", "sur":"vaaada"})
db.name

error: [Errno 111] Connection refused